##  Optimze and a Dense Neural Network for gap filling and feature identification

** With a few tweaks to RepRunner, an LSTM can be run instead

In [11]:
import time
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import combinations
# from functools import partial
from multiprocessing import Pool
from functools import partial
from sklearn import metrics
# from sklearn.model_selection import train_test_split

## Personal Modules
import ReadStandardTimeFill as RSTF
import importlib
import DenseNet as Dense
import MiscFuncs as MF
importlib.reload(Dense)
importlib.reload(RSTF)
importlib.reload(MF)

# %matplotlib inline

%matplotlib notebook
%config IPCompleter.greedy=True

from scipy.optimize import minimize, curve_fit
from scipy.stats import norm
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, WhiteKernel, ConstantKernel
from sklearn.externals import joblib
from matplotlib import cm

from scipy import stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.multicomp import MultiComparison

from ipywidgets import FloatProgress
from IPython.display import display
import os  
import shutil
from keras import backend as K
try:pool.close()
except:pass


def Test(params,X,y,YScaled,XScaled,pool):
    return(np.random.rand(params['K']))


def ModSelect(Scope,Site):
    if Site == 'Illisarvik':
        if Scope == 'Full':
            Model = ['wind_speed','air_pressure','PPFD_Avg','AirTC_Avg','VPD',
                    'Temp','VWC','Sedge','Shrub','Grass','Sparse','Out_of_Basin']
        if Scope == 'Test':
            Model = ['PPFD_Avg','Sedge','VPD','wind_speed']
    if Site == 'FishIsland':
        BaseFactors = []
        if Scope == 'Full':
            Model = ['H','Wind Spd','air pressure','Ta','Rn','PPFD','Rain','Water Table',
            'Ts 2.5 cm','Ts 15 cm','VWC','Active Layer','24H Rain','Wtr Tbl Trnd']
        if Scope == 'Test':
            Model = ['H','Water Table','Wind Spd','Active Layer']
    return(Model)

def Combos(Model,L,factor=None):
    Models=[]
    for c in combinations(Model,L):
        c = list(c)
        if factor is None:
            Models.append(c)
        else:
            for f in factor:
                f = f.split('+')
                if set(f).issubset(set(c)) and c not in Models:
                    Models.append(c)
                    
    print('Models: ',Models)
    return(Models)

def Stats(mse,se,j,i,params):
    df = pd.DataFrame(index = [str(j)+'_'+str(i)],
                      data={'Model':[params['Model']],
                            'MSE':[mse],
                            'Size':j,
                            'Number':i,
                            'SE':[se],
                            'Performance':0})
    return(df)

def t(p,n):
    alpha = 1-p
    df = n-1
    return(stats.t.ppf(alpha,df))


FirstRun = True
# FirstRun = False

MP=True
# MP=False
processes=3
Scope = 'Full'
cwd = os.getcwd()
# for Site in ['Illisarvik','FishIsland']:
Site='Illisarvik'
target='fch4'
T= 1#t(0.05,params['K'])


# Model

In [12]:
if __name__ == '__main__' and FirstRun == True:
    XVarriables=ModSelect(Scope,Site)
    prog = FloatProgress(min=0, max=len(XVarriables)+1,description='Running:') # instantiate the bar
    prog2 = FloatProgress(min=0, max=100,description='Running:') # instantiate the bar
    display(prog) # display the bar
    try:shutil.rmtree(cwd+'/'+Site+'/'+target+'/')
    except:pass
    os.mkdir(cwd+'/'+Site+'/'+target+'/')
    for j in range(1,len(XVarriables)+1):
        if j == 1: Inputs = (Combos(XVarriables,j))
        else: Inputs = (Combos(XVarriables,j,Factors))
        i = 0
        for Input in Inputs:
            params = Dense.Params(Scope,target,MP)
            params['Dpath'] = cwd+'/'+Site+'/'
            params['Spath'] = params['Dpath']+'/'+target+'/'+str(j)+'_'+str(i)+'/'
            try:os.mkdir(params['Spath'])
            except:pass
            params['Sname'] = 'Y_'
            params['Inputs'] = Input
            params['Model'] = '+'.join(params['Inputs'])
            RST = RSTF.ReadStandardTimeFill(params,'ECData.csv')#,resample='2H')
            RST.Scale(params['target'],params['Inputs']) 
            y = RST.y*1.0
            X = RST.X*1.0
            params['N']=int(y.shape[0]/30)
            params['Memory'] = (math.floor(100/params['proc'])- 5/params['proc']) * .01
            Y_hat=[]
            y_true=[]
            X_true=[]
            index=[]
            ones=[]
            print('Testing: ',params['Inputs'])
            p
            if MP == False:
                for k in range(params['K']):
                    results = Dense.Bootstrap(k,params,X,y)
#                     d = input('dook')                    plt.figure()
#                     plt.scatter(RST.YScaled.inverse_transform(results[0]),
#                                RST.YScaled.inverse_transform(results[1]))
                    Y_hat.append(RST.YScaled.inverse_transform(results[0]))
                    y_true.append(RST.YScaled.inverse_transform(results[1]))
                    X_true.append(RST.XScaled.inverse_transform(results[2]))
                    ones.append(results[3])
                    prog2.value=(k+1)/params['k']*100
#                     ones.append(results[4])
            else:
                pool = Pool(processes=processes,maxtasksperchild=75)
                for k,results in enumerate(pool.imap(partial(Dense.Bootstrap,params=params,X=X,y=y),range(params['K']))):

                    Y_hat.append(RST.YScaled.inverse_transform(results[0]))
                    y_true.append(RST.YScaled.inverse_transform(results[1]))
                    X_true.append(RST.XScaled.inverse_transform(results[2]))
                    ones.append(results[3])
                    prog2.value=(k+1)/params['k']*100
#                     ones.append(results[4])
                pool.close()
            Y_hat = np.squeeze(np.asanyarray(Y_hat))
            y_true = np.squeeze(np.asanyarray(y_true))
            X_true = np.asanyarray(X_true)
#             index = np.asanyarray(index)
            ones = np.asanyarray(ones)
            params['Memory'] = .95
            if MP == False:
                for k in range(1):
                     mse,se = Dense.Sort_outputs(k,params,Y_hat,y_true,X_true,ones)
            else:
                pool = Pool(processes=1,maxtasksperchild=75)
                for k,results in enumerate(pool.imap(partial(Dense.Sort_outputs,params=params,
                 Y_hat=Y_hat,y_true=y_true,X_true=X_true,ones=ones),range(1))):
                     mse,se = results
                pool.close()
            if i == 0:Level = Stats(mse,se,j,i,params)
            else:Level = Level.append(Stats(mse,se,j,i,params))
            i += 1
            prog.value=j+i/len(Inputs)
        Min = Level.loc[Level['MSE']==Level['MSE'].min()]
        T= t(0.05,params['K'])
        Factors = Level.loc[Level['MSE']<=Min['MSE'].values[0]+Min['SE'].values[0]*T,'Model'].values
        print(Factors)
        Level.loc[Level['MSE']<=Min['MSE'].values[0]+Min['SE'].values[0]*T,'Performance']=1
        if j == 1:Records = Level
        else:Records = Records.append(Level)

    Records = Records.reset_index(drop=True)
    Min = Records.loc[Records['MSE']==Records['MSE'].min()]
    Records.loc[Records['MSE']<=Min['MSE'].values[0]+Min['SE'].values[0]*T,'Performance']=2
    Records.loc[Records['MSE']==Min['MSE'].values[0],'Performance']=3
    Honorable = Records.loc[Records['Performance']==1]
    Top = Records.loc[Records['Performance']==2]
    Best = Records.loc[Records['Performance']==3]
    plt.figure()
    plt.bar(Honorable.index,Honorable['MSE'],yerr=Honorable['SE'],color='blue')
    plt.bar(Top.index,Top['MSE'],yerr=Top['SE'],color='green')
    plt.bar(Best.index,Best['MSE'],yerr=Best['SE'],color='red')
    Records.drop(Records.columns[Records.columns.str.contains('Unnamed',case = False)],axis = 1)
    Records.to_csv(params['Dpath']+'/'+target+'_Runs.csv')
    FirstRun = False
else:    
    print('OldRun!')
    params = Dense.Params(Scope,target,MP)
    params['Dpath'] = cwd+'/'+Site+'/'
    RST = RSTF.ReadStandardTimeFill(params,'ECData.csv')#,resample='2H')
    params = Dense.Params(Scope,target,MP)
    params['Dpath'] = cwd+'/'+Site+'/'
    Records = pd.read_csv(params['Dpath']+'/'+target+'_Runs.csv',delimiter = ',',header = 0)
    Min = Records.loc[Records['MSE']==Records['MSE'].min()]
    Records.loc[Records['MSE']<=Min['MSE'].values[0]+Min['SE'].values[0]*T,'Performance']=2
    Records.loc[Records['MSE']==Min['MSE'].values[0],'Performance']=3
    T= 1#t(0.05,params['K'])
    Honorable = Records.loc[Records['Performance']==1]
    Top = Records.loc[Records['Performance']==2]
    Best = Records.loc[Records['Performance']==3]
    plt.figure()
    plt.bar(Honorable.index,Honorable['MSE'],yerr=Honorable['SE'],color='blue')
    plt.bar(Top.index,Top['MSE'],yerr=Top['SE'],color='green')
    plt.bar(Best.index,Best['MSE'],yerr=Best['SE'],color='red')
    Records.to_csv(params['Dpath']+'/'+target+'_Runs.csv')
print(Records)
print((Records['MSE']+Records['SE']))

A Jupyter Widget

Models:  [['wind_speed'], ['air_pressure'], ['PPFD_Avg'], ['AirTC_Avg'], ['VPD'], ['Temp'], ['VWC'], ['Sedge'], ['Shrub'], ['Grass'], ['Sparse'], ['Out_of_Basin']]
Testing:  ['wind_speed']


Process ForkPoolWorker-234:
Process ForkPoolWorker-235:
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-236:
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/ubuntu/NetworkAnalysis/DenseNet.py", line 135, in Bootstrap
    Y_hat=Train_DNN(params,X_train,y_train,X,y)
  File "/usr/lib/python3.5/multiprocessing/pool.py", l

KeyboardInterrupt: 

  File "/usr/local/lib/python3.5/dist-packages/keras/engine/training.py", line 1598, in fit
    validation_steps=validation_steps)
  File "/usr/local/lib/python3.5/dist-packages/keras/engine/training.py", line 1183, in _fit_loop
    outs = f(ins_batch)
  File "/usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py", line 2273, in __call__
    **self.session_kwargs)
  File "/usr/local/lib/python3.5/dist-packages/tensorflow/python/client/session.py", line 895, in run
    run_metadata_ptr)
  File "/usr/local/lib/python3.5/dist-packages/tensorflow/python/client/session.py", line 1124, in _run
    feed_dict_tensor, options, run_metadata)
  File "/usr/local/lib/python3.5/dist-packages/tensorflow/python/client/session.py", line 1321, in _do_run
    options, run_metadata)
  File "/usr/local/lib/python3.5/dist-packages/tensorflow/python/client/session.py", line 1327, in _do_call
    return fn(*args)
  File "/usr/local/lib/python3.5/dist-packages/tensorflow/python/client/sess

# Map Results

# Predict

In [ ]:

plt.figure(figsize=(8,7))
Data = Data.sort_values(by=Key)
# Data.index = Data[Key]

Data = Data[np.isfinite(Data['True'])]

plt.scatter(RST.Master[Key],RST.Master[target],edgecolor='black',facecolor='white')
plt.plot(Data[Key],Data[target],
         label= params['target']+' Model\nr^2: '+str(np.round(metrics.r2_score(Data['True'],
                                                                   Data[params['target']])**2,3)))
# plt.plot(Data.index,Data['Var'],label= params['target']+
# ' Model\nRMSE: '+str(np.round(metrics.mean_squared_error(Data['True'],
#                                                                    Data[params['target']])**2,3)))


plt.fill_between(Data[Key], Data[target]-Data['PI'], 
                 Data[target]+Data['PI'],  color = 'green', alpha = 0.4, 
                 label = '95% PI')
plt.fill_between(Data[Key], Data[target]-Data['CI'], 
                 Data[target]+Data['CI'],  color = 'red', alpha = 0.4, 
                 label = '95% CI')
plt.legend()
# print(RST.Master)

plt.figure()

# plt.plot(Data[Key], Data['Var'])

plt.scatter(Data[target],Data['True'])

# print(Data['SE'])